# Degs Annotation Report: Multi-database Annotation Summary

This Jupyter notebook is designed to generate a report file based on annotation results obtained from annotation tools such as BLAST or Diamond. To use this notebook, users must have both the annotation results in TSV format and a table containing additional information about the transcripts (usually obtained with DeSeq2 software). The notebook's intuitive user interface and modular design make it easy to customize the report based on specific research needs. With its ability to quickly and efficiently generate reports from annotation data, this Jupyter notebook is an invaluable tool for researchers working with transcriptomic data.

*Remember:* The tsv files **must** contain at least the following columns: "qseqid", "sseqid", "pident", "slen", "stitle", "length", "evalue".

First of all, run the following cell for defining useful functions and loading the libraries.

In [1]:
#Importing libraries
import pandas as pd
from openpyxl.utils import get_column_letter
import os
import time

# Display all columns
pd.set_option('display.max_columns', None)

# Define useful functions

# Get the transcript IDs from the table
def get_transcripts_from_id(transcripts, table):
    transcripts = transcripts.unique()

    dic = dict()

    for t in transcripts:
        for x in table.transcript:
            if t.startswith(x):
                dic[t] = x
    return dic

# Get the hyperlink from the sseqid
def make_hyperlink(sseqid, database):
    
    try:
        if database.lower() == 'nr':
            protein_accession = sseqid.split(" ")[0]
            url = "https://www.ncbi.nlm.nih.gov/gene/?term={}"
        else:
            protein_accession = sseqid.split("|")[1]
            url = "https://www.uniprot.org/uniprotkb/{}/entry"
    except:
        print(sseqid)
        return ""
        
    return '=HYPERLINK("%s", "%s")' % (url.format(protein_accession), protein_accession)

# Get the accession from the sseqid
def get_accession(sseqid, database):

    if database.lower() == 'nr':
        try:
            return sseqid.split(" ")[0]
        except:
            return ""
    else:
        try:
            return sseqid.split("|")[1]
        except:
            return ""

# Get the scientific name from the stitle
def get_sciname(x, database):

    if database.lower() == 'nr':

        try:
            os_index = - x[::-1].index('[')
        except:
            return ""

        return x[os_index:-1]
    
    else:
        try:
            os_index = x.index('OS=')
            ox_index = x.index('OX=')
        except:
            return ""

        return x[os_index+3:ox_index-1]

# Get the protein function from the stitle
def get_protein_function(x, database):

    s = '[' if database.lower() == 'nr' else 'OS='
    x_l = x.split(" ")

    try:
        nex = ' '.join(x_l[1:x_l.index(next(x for x in x_l if x.startswith(s)))])
    except:
        return ""
    
    return nex

# Get the locus name from the stitle
def get_locus_name(x, database):

    if database.lower() == 'nr':
        return ""
    
    try:
        return x.split("|")[1]
    except:
        return ""

# Get the gene from the stitle
def get_gene(x, database):

    if database.lower() == 'nr':
        return ""

    try:
        gn_index = x.index('GN=')
        pe_index = x.index('PE=')
    except:
        return None
    
    return x[gn_index+3:pe_index-1]

In this section, the user must customize the generation parameters (by appropriately modifying the variables) following the instructions in the comments in the cell below.

So once you have filled it (with your data), exec this cell to setting up the variables.

In [5]:
# Insert the titles of the graph
title = "Blast-culex_pipiens"

# Insert the table (with additional informations) path
table_path = "../data/tables"

# Insert the path of the report (remember the "/" at the end)
out_path = "../results/"

# Set databases names
tool_names = [
        "blastp", 
        "blastx",
]

# Insert the databases names (the order must match the result files order)
databases_names =[
    "Nr", 
    "TrEMBL",
    "Swiss-Prot",
]

# Insert the names (or paths) of the tsv files (remember to follow the same order used when setting the tool_names and databases_names)
files = {

    'Nr':[
        "../data/culex_pipiens-longest_orf-blastp-nr.tsv",
        "../data/culex_pipiens-longest_orf-blastx-nr.tsv",
    ],

    'TrEMBL':[
        "../data/culex_pipiens-longest_orf-blastp-tr.tsv",
        "../data/culex_pipiens-longest_orf-blastx-tr.tsv",
    ],

    'Swiss-Prot':[
        "../data/culex_pipiens-longest_orf-blastp-sp.tsv",
        "../data/culex_pipiens-longest_orf-blastx-sp.tsv"
    ],

}

# Set the outformat
# e.g. 
#outfmt = "qseqid qlen sseqid sallseqid slen qstart qend sstart send qseq full_qseq sseq full_sseq evalue bitscore score length pident nident mismatch positive gapopen gaps ppos qframe btop cigar staxids sscinames sskingdoms skingdoms sphylums stitle salltitles qcovhsp scovhsp qtitle qqual full_qqual qstrand"
#outfmt = "qseqid qlen sseqid sallseqid slen qstart qend sstart send qseq full_qseq sseq full_sseq evalue bitscore score length pident nident mismatch positive gapopen gaps ppos qframe btop cigar stitle salltitles qcovhsp scovhsp qtitle qqual full_qqual qstrand"

# If there are column names in the first row of tsv files then set outfmt = None
outfmt = "qseqid qlen sseqid slen evalue bitscore score length pident stitle"
#outfmt = None

# Default features to extract, you can change them if you want to modify the report
# Columns names (modify this list by inserting the column names of the report)
all_features = ["transcript", "log2FoldChange", "padj", 
            "protein_accession", "sequence_identity", "alignment_length", 
            "evalue", "database", "gene", "locus_name", "sequence_description",
            "sequence_length", "organism", "protein_product"]

features = ["qseqid", "sseqid", "pident", "slen", "stitle", "length", "evalue"]

Now you can run the cell below to define the functions that generate the final dataframe. 

In [6]:
# Define useful functions

def get_dataframe_from_files():

    """
    This function is used to import the whole dataframe from the all tsv files.
    """

    # Define the features of interest
    features = ["qseqid", "sseqid", "pident", "slen", "stitle", "length", "evalue"]

    # Create an empty dataframe
    df = pd.DataFrame()

    # Iterate over the files
    for db in files.keys():

        # Iterate over the tools and files
        for file, tool in zip(files[db], tool_names):

            # Import the tsv dataset.
            # If outfmt = None the first row will be used as column names
            if outfmt == None:
                tmp = pd.read_csv(file, sep="\t", low_memory=False)
            else:
                tmp = pd.read_csv(file, sep="\t", names=outfmt.split(), low_memory=False)

            # Extract the features of interest
            tmp = tmp[features]

            # Add the tool name
            tmp['tool'] = tool
            
            # Add the database name
            tmp['database'] = db
            
            # Concatenate the dataframe with the current one
            df = pd.concat([df, tmp])

    return df

def generate_df(df, table):

    """
    This function is used to generate the super-table merging the information contained in the DeSeq2 table with
    the alignment information inside the complete dataframe.
    """

    # Read the table from DeSeq2 table file
    table = pd.read_csv(table_path + "/" + table, sep='\t')

    # Set the index column name
    table.index.name = 'transcript'
    table.reset_index(inplace=True)

    # Get the relevant transcript IDs from the table
    df['qseqid'] = df['qseqid'].map(get_transcripts_from_id(df['qseqid'], table))

    # For each tool add the respective column and set it to 0 if the used tool is not the current one and 1 otherwise
    for row in tool_names:
        df[row] = [row if x == row else '' for x in df['tool']]
    
    # Group the dataframe by qseqid, sseqid, pident, slen, stitle, length, evalue, database
    df = df.groupby(["qseqid", "sseqid", "pident", "slen", "stitle", "length", "evalue", "database"]).sum().reset_index()
    
    # Drop the row column
    df.drop("tool", axis=1, inplace=True)

    # Rename the columns
    df.rename(columns={'pident': 'sequence_identity',
                       'length': 'alignment_length',
                       'stitle': 'sequence_description',
                       'slen':   'sequence_length'
    }, inplace=True)

    # Add the gene, organism, protein_accession, protein_product, locus_name columns        
    df['gene'] = [get_gene(row['sequence_description'], row['database']) for idx, row in df.iterrows()]
    df['organism'] = [get_sciname(row['sequence_description'], row['database']) for idx, row in df.iterrows()]
    df['protein_accession'] = df.apply(lambda x: make_hyperlink(x.sseqid, x.database), axis=1)
    df['protein_product'] = [get_protein_function(row['sequence_description'], row['database']) for idx, row in df.iterrows()]
    df['locus_name'] = [get_locus_name(row['sseqid'], row['database']) for idx, row in df.iterrows()]

    # Merge the dataframe with the table
    df = pd.merge(df, table, left_on='qseqid', right_on='transcript', how='inner')

    # Sort the dataframe by transcript, evalue
    df.sort_values(['transcript', 'evalue'], inplace=True)

    # Reset the index
    df.reset_index(drop=True, inplace=True)

    # Group the dataframe by transcript and take the top 20 rows
    df = df.groupby('transcript').head(20)

    # Set the duplicated transcript, log2FoldChange and padj to None
    df.loc[df.duplicated(subset=['transcript', 'log2FoldChange', 'padj']), ['transcript', 'log2FoldChange', 'padj']] = None
    
    # Reorder the columns
    df = df[["transcript"] + tool_names + all_features[1:]]

    return df

def generate_xlsx(df, path, table):

    """
    This function is used to generate the xlsx file with the super-table.
    """

    # Create a Pandas Excel writer using XlsxWriter as the engine
    df_writer = pd.ExcelWriter(path + "_" + table.split("/")[-1] + '.xlsx') 

    # Write the DataFrame to the working ExcelWriter
    df.to_excel(df_writer, sheet_name='report', index=False)

    # Get the xlsxwriter workbook and worksheet objects
    worksheet = df_writer.sheets['report']

    # Set the column width
    for column in range(df.shape[1]):

        # Get the max length of the columns
        column_length = max(df.iloc[:, column].astype(str).map(len).max(), len(df.columns[column])) + 3

        # Setting the column length
        column_letter = get_column_letter(column + 1)

        # Adjust the column width
        worksheet.column_dimensions[column_letter].width = column_length

    # Close the Pandas Excel writer and output the Excel file
    df_writer.close()

Now run the cell below to create a super-table excel file for each table file in the table directory.

In [7]:
# Save the super-table for each table in the tables directory

# Import the whole dataframe from the tsv files
df = get_dataframe_from_files()

for table_file in os.listdir(table_path):

    # Create a copy of the dataframe
    tmp = df.copy()
    
    print("Analysing", table_file[:-4], "table...")

    # get initial time
    start_time = time.time()

    # Generate the dataframe from 
    tmp = generate_df(tmp, table_file)

    # Generate the xlsx file
    generate_xlsx(tmp, out_path + title.replace(" ", "_"), table_file[:-4])

    print("Table", table_file, "done in", "--- %s seconds ---" % (time.time() - start_time))

    # Delete the copy of the dataframe
    del tmp

Analysing culex_pipiens-table-1 table...
Table culex_pipiens-table-1.txt done in --- 0.0628821849822998 seconds ---
Analysing culex_pipiens-table-2 table...
Table culex_pipiens-table-2.txt done in --- 0.0488584041595459 seconds ---
